In [1]:
import numpy as np
import scipy
import seaborn as sns
import datasets
import torch
import evaluate
from tqdm.notebook import tqdm
import os
os.environ['WANDB_DISABLED'] = 'true'
# import wandb

import utils

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments

2023-02-05 19:23:06.668773: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64
2023-02-05 19:23:06.668820: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' 

In [3]:
train_ds, valid_ds, test_ds = utils.load_dataset()

Found cached dataset civil_comments (/home/johnny/.cache/huggingface/datasets/civil_comments/default/0.9.0/e7a3aacd2ab7d135fa958e7209d10b1fa03807d44c486e3c34897aa08ea8ffab)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached shuffled indices for dataset at /home/johnny/.cache/huggingface/datasets/civil_comments/default/0.9.0/e7a3aacd2ab7d135fa958e7209d10b1fa03807d44c486e3c34897aa08ea8ffab/cache-01f767df1b53d154.arrow
Loading cached processed dataset at /home/johnny/.cache/huggingface/datasets/civil_comments/default/0.9.0/e7a3aacd2ab7d135fa958e7209d10b1fa03807d44c486e3c34897aa08ea8ffab/cache-52cd85a4b8b2e0c8.arrow
Loading cached processed dataset at /home/johnny/.cache/huggingface/datasets/civil_comments/default/0.9.0/e7a3aacd2ab7d135fa958e7209d10b1fa03807d44c486e3c34897aa08ea8ffab/cache-9e6abfbab2489ee3.arrow


In [4]:
'''
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
def tokenize_function(examples):
    return tokenizer(examples["text"], padding='do_not_pad')

tokenized_ds = ds.map(tokenize_function, batched=True)
lengths = np.array([len(i['input_ids']) for i in tokenized_ds['validation']])
np.quantile(lengths, [0.90]) = 168
'''

'\ntokenizer = AutoTokenizer.from_pretrained("roberta-base")\ndef tokenize_function(examples):\n    return tokenizer(examples["text"], padding=\'do_not_pad\')\n\ntokenized_ds = ds.map(tokenize_function, batched=True)\nlengths = np.array([len(i[\'input_ids\']) for i in tokenized_ds[\'validation\']])\nnp.quantile(lengths, [0.90]) = 168\n'

In [5]:
labels = np.array(train_ds['label'])
num_minority = np.sum(labels == 1)

np.random.seed(0)

indexes = []
for label in [0, 1]:
    idx = np.arange(len(labels))
    idx = idx[labels == label]
    
    print(len(idx))
    sample = np.random.choice(idx, size=num_minority, replace=False)
    indexes.append(sample)

94163
5837


In [6]:
indexes = np.concatenate(indexes)
train_ds = train_ds.select(indexes)
train_ds

Dataset({
    features: ['text', 'toxicity', 'severe_toxicity', 'obscene', 'threat', 'insult', 'identity_attack', 'sexual_explicit', 'label'],
    num_rows: 11674
})

In [7]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", max_length=168, truncation=True)

tokenized_train = train_ds.map(tokenize_function, batched=True)
tokenized_valid = valid_ds.map(tokenize_function, batched=True)

Loading cached processed dataset at /home/johnny/.cache/huggingface/datasets/civil_comments/default/0.9.0/e7a3aacd2ab7d135fa958e7209d10b1fa03807d44c486e3c34897aa08ea8ffab/cache-8c8361aa1875c2b5.arrow
Loading cached processed dataset at /home/johnny/.cache/huggingface/datasets/civil_comments/default/0.9.0/e7a3aacd2ab7d135fa958e7209d10b1fa03807d44c486e3c34897aa08ea8ffab/cache-a629212e3e74f41d.arrow


In [8]:
tokenized_train.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
np.mean(tokenized_train['label'].numpy())

0.5

In [9]:
tokenized_valid.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
np.mean(tokenized_valid['label'].numpy())

0.0524

In [10]:
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'classifier

In [11]:
training_args = TrainingArguments(output_dir="trainer",
                                  max_steps=1000,
                                  per_device_train_batch_size=32,
                                  evaluation_strategy='steps',
                                  eval_steps=100,
                                  save_strategy='no',
                                  num_train_epochs=1,
                                 )

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [12]:
metric = evaluate.load("roc_auc")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    batch_scores = scipy.special.softmax(logits, axis=-1)
    return metric.compute(prediction_scores=batch_scores[:,1], references=labels)

In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    compute_metrics=compute_metrics,
)

max_steps is given, it will override any value given in num_train_epochs


In [14]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: severe_toxicity, obscene, text, identity_attack, sexual_explicit, toxicity, threat, insult. If severe_toxicity, obscene, text, identity_attack, sexual_explicit, toxicity, threat, insult are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/home/johnny/.conda/envs/torch/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 11674
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimi

Step,Training Loss,Validation Loss,Roc Auc
100,No log,0.457110,0.941122
200,No log,0.315010,0.956806
300,No log,0.234920,0.959931
400,No log,0.288535,0.961854
500,0.292100,0.424403,0.964520
600,0.292100,0.275101,0.961212
700,0.292100,0.295045,0.961509
800,0.292100,0.450668,0.959754
900,0.292100,0.405156,0.959234
1000,0.118000,0.416474,0.959671


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: severe_toxicity, obscene, text, identity_attack, sexual_explicit, toxicity, threat, insult. If severe_toxicity, obscene, text, identity_attack, sexual_explicit, toxicity, threat, insult are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: severe_toxicity, obscene, text, identity_attack, sexual_explicit, toxicity, threat, insult. If severe_toxicity, obscene, text, identity_attack, sexual_explicit, toxicity, threat, insult are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples 

TrainOutput(global_step=1000, training_loss=0.2050847282409668, metrics={'train_runtime': 412.6698, 'train_samples_per_second': 77.544, 'train_steps_per_second': 2.423, 'total_flos': 1390386095505792.0, 'train_loss': 0.2050847282409668, 'epoch': 2.74})

In [15]:
def _evaluate(model, val_dataloader):
    model.eval()
    val_pbar = tqdm(total=len(val_dataloader))

    scores = []
    for batch in val_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        labels = batch['label']
        del(batch['label'])
        with torch.no_grad():
            outputs = model(**batch)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1).cpu().tolist()

        batch_scores = torch.nn.Softmax()(logits)[:,-1]
        scores.append(batch_scores)

        metric.add_batch(prediction_scores=batch_scores, references=labels)
        val_pbar.update(1)
    eval_dict = metric.compute()
    val_pbar.set_description('Roc_auc: %.2f' % eval_dict['roc_auc'])

    eval_dict['predictions'] = torch.cat(scores)

    return eval_dict

In [ ]:
tokenized_test = test_ds.map(tokenize_function, batched=True)
tokenized_test.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

val_dataloader = torch.utils.data.DataLoader(tokenized_test, shuffle=False, batch_size=16, pin_memory=True)
eval_dict = _evaluate(model, val_dataloader)

  0%|          | 0/1705 [00:00<?, ?ba/s]

  0%|          | 0/106555 [00:00<?, ?it/s]

/tmp/ipykernel_31751/1939303410.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  batch_scores = torch.nn.Softmax()(logits)[:,-1]


In [ ]:
predictions = eval_dict['predictions'].cpu().numpy()
np.save('scores/distilbert_balanced', predictions)

In [ ]:
trainer.save_model('./distilbert_balanced')